<a href="https://colab.research.google.com/github/piusAI/Pytorch_Study/blob/main/05_buildmodel_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

[Learn the Basics](intro.html) \|\|
[Quickstart](quickstart_tutorial.html) \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| **Build Model** \|\|
[Autograd](autogradqs_tutorial.html) \|\|
[Optimization](optimization_tutorial.html) \|\| [Save & Load
Model](saveloadrun_tutorial.html)

Build the Neural Network
========================

Neural networks comprise of layers/modules that perform operations on
data. The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace
provides all the building blocks you need to build your own neural
network. Every module in PyTorch subclasses the
[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
A neural network is a module itself that consists of other modules
(layers). This nested structure allows for building and managing complex
architectures easily.

In the following sections, we\'ll build a neural network to classify
images in the FashionMNIST dataset.


In [3]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Get Device for Training
=======================

We want to be able to train our model on an
[accelerator](https://pytorch.org/docs/stable/torch.html#accelerators)
such as CUDA, MPS, MTIA, or XPU. If the current accelerator is
available, we will use it. Otherwise, we use the CPU.


In [4]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"now, I'm Using {device} device!")

now, I'm Using cpu device!


Define the Class
================

We define our neural network by subclassing `nn.Module`, and initialize
the neural network layers in `__init__`. Every `nn.Module` subclass
implements the operations on input data in the `forward` method.


In [5]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),

        )
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

We create an instance of `NeuralNetwork`, and move it to the `device`,
and print its structure.


In [34]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model\'s
`forward`, along with some [background
operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
Do not call `model.forward()` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0
corresponding to each output of 10 raw predicted values for each class,
and dim=1 corresponding to the individual values of each output. We get
the prediction probabilities by passing it through an instance of the
`nn.Softmax` module.


In [38]:
X= torch.rand(1, 28, 28, device = device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f'predicted class : {y_pred}')
# print(next(iter(X)))

predicted class : tensor([0])


Function chaining / Function Instantiation and Immediate call.

In [39]:
softmax_fn = nn.Softmax(dim=1) # like function
pred_probab2 = softmax_fn(logits)

pred_probab3 = nn.Softmax(dim=1)(logits) #collable object for function

print(pred_probab2,"\n")
print(pred_probab3)

tensor([[0.1048, 0.0948, 0.1035, 0.1036, 0.1012, 0.0944, 0.0998, 0.1000, 0.1041,
         0.0938]], grad_fn=<SoftmaxBackward0>) 

tensor([[0.1048, 0.0948, 0.1035, 0.1036, 0.1012, 0.0944, 0.0998, 0.1000, 0.1041,
         0.0938]], grad_fn=<SoftmaxBackward0>)


------------------------------------------------------------------------


Model Layers
============

Let\'s break down the layers in the FashionMNIST model. To illustrate
it, we will take a sample minibatch of 3 images of size 28x28 and see
what happens to it as we pass it through the network.


In [42]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten
==========

We initialize the
[nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
layer to convert each 2D 28x28 image into a contiguous array of 784
pixel values ( the minibatch dimension (at dim=0) is maintained).


In [43]:
flatten= nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear
=========

The [linear
layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
is a module that applies a linear transformation on the input using its
stored weights and biases.


In [45]:
layer1 = nn.Linear(in_features = 28*28, out_features=20)
hidden2= layer1(flat_image)
print(hidden2.size())

torch.Size([3, 20])


In [50]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU
=======

Non-linear activations are what create the complex mappings between the
model\'s inputs and outputs. They are applied after linear
transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use
[nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)
between our linear layers, but there\'s other activations to introduce
non-linearity in your model.


In [48]:
print(f'Before Relu: {hidden2}\n\n')
hidden2= nn.ReLU()(hidden2)
print(f'After hidden2 : {hidden2}')

Before Relu: tensor([[0.0992, 0.3542, 0.0000, 0.3018, 0.0133, 0.1159, 0.0000, 0.0994, 0.0000,
         0.1596, 0.4586, 0.0000, 0.2243, 0.0000, 0.0000, 0.0000, 0.7393, 0.1128,
         0.3692, 0.0000],
        [0.0000, 0.5351, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.3556, 0.0000, 0.0082, 0.0000, 0.0000, 0.0471, 0.6054, 0.0000,
         0.3512, 0.0000],
        [0.1451, 0.3960, 0.0000, 0.0852, 0.0543, 0.2131, 0.0425, 0.0000, 0.0000,
         0.0405, 0.1910, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1559, 0.0000,
         0.2441, 0.1295]], grad_fn=<ReluBackward0>)


After hidden2 : tensor([[0.0992, 0.3542, 0.0000, 0.3018, 0.0133, 0.1159, 0.0000, 0.0994, 0.0000,
         0.1596, 0.4586, 0.0000, 0.2243, 0.0000, 0.0000, 0.0000, 0.7393, 0.1128,
         0.3692, 0.0000],
        [0.0000, 0.5351, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.3556, 0.0000, 0.0082, 0.0000, 0.0000, 0.0471, 0.6054, 0.0000,
         0.3512, 0.0000],
   

In [51]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1219,  0.0027,  0.6980,  0.0641, -0.0608,  0.5037,  0.1769, -0.2377,
          0.3596, -0.4721, -0.5601, -0.1303,  0.4469,  0.0449,  0.1285, -0.6085,
          0.0067, -0.3943,  0.7257,  0.4164],
        [-0.0688, -0.2017,  0.4858,  0.0978, -0.1242, -0.1835, -0.0718, -0.4156,
          0.5168, -0.0669, -0.5403, -0.0252,  0.1392, -0.3674, -0.0937, -0.4059,
         -0.1781, -0.8695,  0.5547,  0.1174],
        [-0.2015,  0.0319,  0.8484, -0.1292,  0.1024, -0.0471,  0.3901, -0.1666,
          0.1620, -0.2393, -0.1767, -0.2881,  0.0476, -0.0687,  0.0408, -0.1686,
         -0.6108, -0.5176,  0.6932,  0.2151]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1219, 0.0027, 0.6980, 0.0641, 0.0000, 0.5037, 0.1769, 0.0000, 0.3596,
         0.0000, 0.0000, 0.0000, 0.4469, 0.0449, 0.1285, 0.0000, 0.0067, 0.0000,
         0.7257, 0.4164],
        [0.0000, 0.0000, 0.4858, 0.0978, 0.0000, 0.0000, 0.0000, 0.0000, 0.5168,
         0.0000, 0.0000, 0.0000, 0.1392, 0.0000, 0.00

nn.Sequential
=============

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)
is an ordered container of modules. The data is passed through all the
modules in the same order as defined. You can use sequential containers
to put together a quick network like `seq_modules`.


In [52]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

nn.Softmax
==========

The last linear layer of the neural network returns [logits]{.title-ref}
- raw values in \[-infty, infty\] - which are passed to the
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)
module. The logits are scaled to values \[0, 1\] representing the
model\'s predicted probabilities for each class. `dim` parameter
indicates the dimension along which the values must sum to 1.


In [55]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

Model Parameters
================

Many layers inside a neural network are *parameterized*, i.e. have
associated weights and biases that are optimized during training.
Subclassing `nn.Module` automatically tracks all fields defined inside
your model object, and makes all parameters accessible using your
model\'s `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter, and print its size and
a preview of its values.


In [58]:
print(f'model structure:{model}\n\n')
for name, param in model.named_parameters():
  print(f"Layer : {name} | Size : {param.size()} | Values : {param[:2]} \n" )

model structure:NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer : linear_relu_stack.0.weight | Size : torch.Size([512, 784]) | Values : tensor([[ 0.0250, -0.0035, -0.0082,  ..., -0.0157,  0.0151, -0.0220],
        [-0.0350, -0.0298,  0.0070,  ...,  0.0065,  0.0201,  0.0119]],
       grad_fn=<SliceBackward0>) 

Layer : linear_relu_stack.0.bias | Size : torch.Size([512]) | Values : tensor([ 0.0080, -0.0237], grad_fn=<SliceBackward0>) 

Layer : linear_relu_stack.2.weight | Size : torch.Size([512, 512]) | Values : tensor([[-0.0165, -0.0111,  0.0337,  ...,  0.0002,  0.0169, -0.0095],
        [ 0.0397,  0.0345,  0.0301,  ..., -0.0275, -0.0244, -0.0164]],
       grad_fn=<SliceBackward0>) 

Layer : linear_relu_stack.2.b

like check The parameter
Weight, bias

------------------------------------------------------------------------


Further Reading
===============

-   [torch.nn API](https://pytorch.org/docs/stable/nn.html)
